#### Importing necessary libraries

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None # This avoids SettingWithCopyWarnings
import numpy as np



In [3]:
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

### Initializing Variables


In [4]:
data={} # Dictionary to contain pandas dataframe for all the stocks. This is to avoid creating variable for each stock 
        # to store data
finalData={} # This should contain our final output and that is Renko OHLC data
n=7 # Period for ATR
renkoData={} # It contains information on the lastest bar of renko data for the number of stocks we are working on

### Average True Range is generally used as Renko bar size. However we can also use a custom bar size

In [5]:
def ATR(df,n): #df is the DataFrame, n is the period 7,14 ,etc
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1)
    df['ATR']=np.nan
    df.ix[n-1,'ATR']=df['TR'][:n-1].mean() #.ix is deprecated from pandas version- 0.19
    for i in range(n,len(df)):
        df['ATR'][i]=(df['ATR'][i-1]*(n-1)+ df['TR'][i])/n
    return 

### Set of Stocks

In [6]:
StockList=['INFY', 'ICICIBANK'] 

### Fetching data using Yahoo API,  Setting up Pandas Dataframe in the dictionary 'data' with key as Stock name and Value as DataFrame


In [7]:
start='2016-1-1'
for stock in StockList:
    data[stock]=pdr.get_data_yahoo(stock+'.NS', start)

[*********************100%***********************]  1 of 1 downloaded

#### Sometimes due to rate limiting factor, Yahoo api when called yeilds no data. Below cell helps us out in figuring whether we need to ping the api again. 

In [8]:
for stock in data:
    if data[stock].empty:
        print stock

In [9]:
for stock in data:
    print stock, data[stock].head()

ICICIBANK                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2016-01-01  237.544998  239.636002  234.544998  239.091003  229.999863   
2016-01-04  237.272995  237.591003  231.500000  232.317993  223.484390   
2016-01-05  232.955002  234.091003  228.817993  233.363998  224.490616   
2016-01-06  232.182007  233.408997  226.544998  227.363998  218.718781   
2016-01-07  224.000000  225.182007  221.000000  224.317993  215.788589   

              Volume  
Date                  
2016-01-01   5998097  
2016-01-04   9435792  
2016-01-05   8966978  
2016-01-06  17416181  
2016-01-07  18240713  
INFY                    Open         High          Low        Close    Adj Close  \
Date                                                                          
2016-01-01  1100.000000  1108.949951  1093.199951  1105.250000  1049.958252   
2016-01-04  1099.949951  1102.449951  1076.050049  1078.900024  1024

###  Data Cleaning, Column renaming, etc

In [10]:
for stock in data:
    data[stock].drop(data[stock][data[stock].Volume == 0].index, inplace=True) # Data Cleaning
    ATR(data[stock],n)
    data[stock]=data[stock][['Open','High','Low','Close','ATR']] # Removing unwanted columns

In [11]:
data['INFY'].head() #Since ATR period is 7, we will see atr column filling out from 7th row

,Open,High,Low,Close,ATR
Date,,,,,
2016-01-01,1100.000000,1108.949951,1093.199951,1105.250000,NaN
2016-01-04,1099.949951,1102.449951,1076.050049,1078.900024,NaN
2016-01-05,1085.500000,1085.500000,1062.250000,1074.050049,NaN
2016-01-06,1074.650024,1074.650024,1058.900024,1069.349976,NaN
2016-01-07,1060.050049,1067.900024,1047.900024,1050.800049,NaN


In [12]:
data['ICICIBANK'].head(10)

,Open,High,Low,Close,ATR
Date,,,,,
2016-01-01,237.544998,239.636002,234.544998,239.091003,NaN
2016-01-04,237.272995,237.591003,231.500000,232.317993,NaN
2016-01-05,232.955002,234.091003,228.817993,233.363998,NaN
2016-01-06,232.182007,233.408997,226.544998,227.363998,NaN
2016-01-07,224.000000,225.182007,221.000000,224.317993,NaN
2016-01-08,222.272995,225.272995,221.317993,222.772995,NaN
2016-01-11,219.908997,221.727005,216.408997,217.682007,5.856336
2016-01-12,219.091003,219.863998,214.182007,215.227005,5.831430
2016-01-13,216.182007,220.817993,210.091003,217.727005,6.530795


### Initializing Renkodata Dictionary which will contain Renko BrickSize, Open, Close and its Color

In [13]:
for stock in data:
    renkoData[stock]={'BrickSize':0.0, 'Open':0.0,'Close':0.0,'Color':''}

### Setting up Bricksize from the last row of ATR column or it can be set manually as well with custom size 

In [14]:
for stock in data:
    renkoData[stock]['BrickSize']=round(data[stock]['ATR'][-1],2) #This can be set manually as well!
    renkoData[stock]['Open']=renkoData[stock]['BrickSize']+renkoData[stock]['Close'] # This can be done the otherway round
                                                                                    # as well.'Close' = 'BrickSize' - 'Open' 
    renkoData[stock]['Color']='red'    # Should you choose to do the other way round, please change the color to 'green'

In [15]:
renkoData['ICICIBANK']

{'BrickSize': 7.73, 'Close': 0.0, 'Color': 'red', 'Open': 7.73}

In [16]:
renkoData['INFY']

{'BrickSize': 22.5, 'Close': 0.0, 'Color': 'red', 'Open': 22.5}

 ### Setting us the finalData dictionary which will contain  Renko OHLC data along with the bar color!

In [17]:
for stock in data:
    finalData[stock]=pd.DataFrame()
    finalData[stock].index.name='Date'
    finalData[stock]['ReOpen']=0.0
    finalData[stock]['ReHigh']=0.0
    finalData[stock]['ReLow']=0.0
    finalData[stock]['ReClose']=0.0
    finalData[stock]['Color']=''


### The calculation involves searching for Renko bars as and when prices close above or below the previous renko bars and recording it 

In [18]:
for stock in data: # This loops thorugh all the stocks in the data dictionary
    for index,row in data[stock].iterrows(): # One may choose to use Pure python instead of Iterrows to loop though each n 
                                         # every row to improve performace if datasets are large.
        if renkoData[stock]['Open']> renkoData[stock]['Close']: 
            while row['Close']> renkoData[stock]['Open']+renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']+=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']+=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Close']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['Color'].loc[index]='green'

            while row['Close'] < renkoData[stock]['Close']-renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']-=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']-=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Open']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['Color'].loc[index]='red'
                
        else:
            while row['Close']< renkoData[stock]['Open']-renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']-=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']-=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Close']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['Color'].loc[index]='red'
                
            while row['Close'] > renkoData[stock]['Close']+renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']+=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']+=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Open']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['Color'].loc[index]='green'



In [19]:
finalData['ICICIBANK'].tail()

,ReOpen,ReHigh,ReLow,ReClose,Color
2018-03-01,316.93,316.93,309.20,309.20,red
2018-03-06,309.20,309.20,301.47,301.47,red
2018-03-07,301.47,301.47,293.74,293.74,red
2018-03-22,293.74,293.74,286.01,286.01,red
2018-03-23,286.01,286.01,278.28,278.28,red


In [20]:
finalData['INFY'].tail()

,ReOpen,ReHigh,ReLow,ReClose,Color
2018-01-23,1147.5,1170.0,1147.5,1170.0,green
2018-02-06,1147.5,1147.5,1125.0,1125.0,red
2018-02-14,1125.0,1125.0,1102.5,1102.5,red
2018-02-22,1125.0,1147.5,1125.0,1147.5,green
2018-02-28,1147.5,1170.0,1147.5,1170.0,green


#### Taking data to excel sheet

In [21]:
writer = pd.ExcelWriter('Data.xlsx')
for stock in data:
    data[stock].to_excel(writer,stock)
writer.save()

writer = pd.ExcelWriter('RenkoData.xlsx')
for stock in finalData:
    finalData[stock].to_excel(writer,stock)
writer.save()
